Initial API test

In [114]:
import requests
import sqlite3
import time


lat = '51.5072'
lon = '0.1276'
now = '1708622726'#int(time.time())
from key import key

link = f'https://api.openweathermap.org/data/3.0/onecall/timemachine?lat={lat}&lon={lon}&dt={now}&appid={key}&units=metric'
conn = sqlite3.connect('weather.db')
response = requests.get(link)
response.text

'{"lat":51.5072,"lon":0.1276,"timezone":"Europe/London","timezone_offset":0,"data":[{"dt":1708622726,"sunrise":1708585263,"sunset":1708622726,"temp":4.83,"feels_like":-0.57,"pressure":981,"humidity":90,"dew_point":3.33,"uvi":0,"clouds":100,"visibility":10000,"wind_speed":9.77,"wind_deg":280,"weather":[{"id":500,"main":"Rain","description":"light rain","icon":"10n"}],"rain":{"1h":1}}]}'

In [160]:
data = response.json()
data

{'lat': 51.5072,
 'lon': 0.1276,
 'timezone': 'Europe/London',
 'timezone_offset': 0,
 'data': [{'dt': 1708622726,
   'sunrise': 1708585263,
   'sunset': 1708622726,
   'temp': 4.83,
   'feels_like': -0.57,
   'pressure': 981,
   'humidity': 90,
   'dew_point': 3.33,
   'uvi': 0,
   'clouds': 100,
   'visibility': 10000,
   'wind_speed': 9.77,
   'wind_deg': 280,
   'weather': [{'id': 500,
     'main': 'Rain',
     'description': 'light rain',
     'icon': '10n'}],
   'rain': {'1h': 1}}]}

Cleaning the response data

Initially I didn't realise that the 'rain' and 'snow' data were given as dict because they are not always included.

This caused an SQLite dict error later, so I had to add in a cleaning step for this.

In [157]:
d1 = {'lat': data['lat'], 'lon': data['lon'], 'timezone': data['timezone'], 'timezone_offset': data['timezone_offset']}

for i in data['data']:
    d1.update(i)

for i in d1['weather']:
    d1.update(i)

d1.pop('weather')

data = d1

#if 'rain' in data.keys():
    #data['rain'] = list(data['rain'].values())[0]

if 'rain' in data.keys():
    data['rain'] = [*data['rain'].values()][0]

if 'snow' in data.keys():
    data['snow'] = [*data['snow'].values()][0]

data

{'lat': 51.5072,
 'lon': 0.1276,
 'timezone': 'Europe/London',
 'timezone_offset': 0,
 'dt': 1708622726,
 'sunrise': 1708585263,
 'sunset': 1708622726,
 'temp': 4.83,
 'feels_like': -0.57,
 'pressure': 981,
 'humidity': 90,
 'dew_point': 3.33,
 'uvi': 0,
 'clouds': 100,
 'visibility': 10000,
 'wind_speed': 9.77,
 'wind_deg': 280,
 'rain': 1,
 'id': 500,
 'main': 'Rain',
 'description': 'light rain',
 'icon': '10n'}

To avoid SQL insertion errors due to lack of data values, I added a step to include placeholders.

This is necessary because the API does not send keys with null values.

In [179]:
all_api_keys = ['lat', 'lon', 'timezone', 'timezone_offset', 'dt', 
                'sunrise', 'sunset', 'temp', 'feels_like', 'pressure', 
                'humidity', 'dew_point', 'uvi', 'clouds', 'visibility', 
                'wind_speed', 'wind_deg', 'wind_gust', 'id', 'main', 
                'description', 'icon', 'rain', 'snow']

for i in all_api_keys:
    if i not in data.keys():
        i = {i: 'N/A'}
        data.update(i)

data

{'lat': 51.5072,
 'lon': 0.1276,
 'timezone': 'Europe/London',
 'timezone_offset': 0,
 'dt': 1708622726,
 'sunrise': 1708585263,
 'sunset': 1708622726,
 'temp': 4.83,
 'feels_like': -0.57,
 'pressure': 981,
 'humidity': 90,
 'dew_point': 3.33,
 'uvi': 0,
 'clouds': 100,
 'visibility': 10000,
 'wind_speed': 9.77,
 'wind_deg': 280,
 'rain': 1,
 'id': 500,
 'main': 'Rain',
 'description': 'light rain',
 'icon': '10n',
 'wind_gust': 'N/A',
 'snow': 'N/A'}

Placing all cleaning code into a function

In [165]:
def clean_data(data):

    d1 = {'lat': data['lat'], 'lon': data['lon'], 'timezone': data['timezone'], 'timezone_offset': data['timezone_offset']}

    for i in data['data']:
        d1.update(i)

    for i in d1['weather']:
        d1.update(i)

    d1.pop('weather')

    data = d1

    if 'rain' in data.keys():
        data['rain'] = [*data['rain'].values()][0]

    if 'snow' in data.keys():
        data['snow'] = [*data['snow'].values()][0]

    all_api_keys = ['lat', 'lon', 'timezone', 'timezone_offset', 'dt', 
                'sunrise', 'sunset', 'temp', 'feels_like', 'pressure', 
                'humidity', 'dew_point', 'uvi', 'clouds', 'visibility', 
                'wind_speed', 'wind_deg', 'wind_gust', 'id', 'main', 
                'description', 'icon', 'rain', 'snow']
    
    for i in all_api_keys:
        if i not in data.keys():
            i = {i: 'N/A'}
            data.update(i)

    return data

data = clean_data(data)

data

{'lat': 51.5072,
 'lon': 0.1276,
 'timezone': 'Europe/London',
 'timezone_offset': 0,
 'dt': 1708622726,
 'sunrise': 1708585263,
 'sunset': 1708622726,
 'temp': 4.83,
 'feels_like': -0.57,
 'pressure': 981,
 'humidity': 90,
 'dew_point': 3.33,
 'uvi': 0,
 'clouds': 100,
 'visibility': 10000,
 'wind_speed': 9.77,
 'wind_deg': 280,
 'rain': 1,
 'id': 500,
 'main': 'Rain',
 'description': 'light rain',
 'icon': '10n',
 'wind_gust': 'N/A',
 'snow': 'N/A'}

Creating SQL table and loading values

In [182]:
conn = sqlite3.connect('weather.db')
cursor = conn.cursor()

cursor.executescript('''

        CREATE TABLE IF NOT EXISTS weather (
                    lat,
                    lon,
                    timezone,
                    timezone_offset,
                    date,
                    sunrise,
                    sunset,
                    temp,
                    feels_like,
                    pressure,
                    humidity,
                    dew_point,
                    uvi,
                    clouds,
                    visibility,
                    wind_speed,
                    wind_gust,
                    wind_deg,
                    weather_id,
                    main,
                    description,
                    icon,
                    rain,
                    snow     
        )
    ''')

cursor.execute('''INSERT INTO weather VALUES (:lat, :lon, :timezone, :timezone_offset,
               :dt, :sunrise, :sunset, :temp, :feels_like, :pressure, :humidity,
               :dew_point, :uvi, :clouds, :visibility, :wind_speed, :wind_gust, :wind_deg,
               :id, :main, :description, :icon, :rain, :snow)''', data
               )

conn.commit()

Quick test to see that everything is working so far

In [170]:
import pandas as pd

df = pd.read_sql_query('''SELECT * FROM weather''', conn)
df

,lat,lon,timezone,timezone_offset,date,sunrise,sunset,temp,feels_like,pressure,...,visibility,wind_speed,wind_gust,wind_deg,weather_id,main,description,icon,rain,snow
0,51.5072,0.1276,Europe/London,0,1708622726,1708585263,1708622726,4.83,-0.57,981,...,10000,9.77,N/A,280,500,Rain,light rain,10n,1,N/A


Creating drop table function

In [183]:
def drop_table():

    cursor.executescript('''

        DROP TABLE IF EXISTS weather

    ''')

    conn.commit()

drop_table()

Adding a last executed file so the database can be updated efficiently

In [87]:
try:
    with open('last_executed.txt') as f:
        last_execution_date = int(f.read())
except FileNotFoundError:
        last_execution_date = int(time.time())

last_execution_date

1708583200

In [80]:
with open('last_executed.txt', 'w') as f:
            f.write(str(int(time.time())))

Creating function to build a list of links for every hour since the code was last executed

In [109]:
def get_links():
    
    if last_execution_date // 86400 ==  time.time() // 86400:
        return []

    links = []

    def build_link(date):
        lat = '51.5072'
        lon = '0.1276'
        from key import key
        link = f'https://api.openweathermap.org/data/3.0/onecall/timemachine?lat={lat}&lon={lon}&dt={date}&appid={key}&units=metric'
        links.append(link)

    nearest_hour = int(time.time() - time.time() % 3600)
    nearest_hour_old = last_execution_date - last_execution_date % 3600

    for x in range(nearest_hour_old, nearest_hour, 3600):
        build_link(x)
    
    return links

get_links()

['https://api.openweathermap.org/data/3.0/onecall/timemachine?lat=51.5072&lon=0.1276&dt=1708581600&appid=e276140c3e33a20f93a7c86e198a285e&units=metric',
 'https://api.openweathermap.org/data/3.0/onecall/timemachine?lat=51.5072&lon=0.1276&dt=1708585200&appid=e276140c3e33a20f93a7c86e198a285e&units=metric',
 'https://api.openweathermap.org/data/3.0/onecall/timemachine?lat=51.5072&lon=0.1276&dt=1708588800&appid=e276140c3e33a20f93a7c86e198a285e&units=metric',
 'https://api.openweathermap.org/data/3.0/onecall/timemachine?lat=51.5072&lon=0.1276&dt=1708592400&appid=e276140c3e33a20f93a7c86e198a285e&units=metric',
 'https://api.openweathermap.org/data/3.0/onecall/timemachine?lat=51.5072&lon=0.1276&dt=1708596000&appid=e276140c3e33a20f93a7c86e198a285e&units=metric',
 'https://api.openweathermap.org/data/3.0/onecall/timemachine?lat=51.5072&lon=0.1276&dt=1708599600&appid=e276140c3e33a20f93a7c86e198a285e&units=metric',
 'https://api.openweathermap.org/data/3.0/onecall/timemachine?lat=51.5072&lon=0.12

In [176]:
conn = sqlite3.connect('weather.db')
cursor = conn.cursor()

def extract_load(links):

    for link in links:

        response = requests.get(link)

        data = clean_data(response.json())

        cursor.executescript('''

            CREATE TABLE IF NOT EXISTS weather (
                     lat,
                     lon,
                     timezone,
                     timezone_offset,
                     date_time,
                     sunrise,
                     sunset,
                     temp,
                     feels_like,
                     pressure,
                     humidity,
                     dew_point,
                     uvi,
                     clouds,
                     visibility,
                     wind_speed,
                     wind_gust,
                     wind_deg,
                     weather_id,
                     main,
                     description,
                     icon,
                     rain,
                     snow     
            )
        ''')

        cursor.execute('''INSERT INTO weather VALUES (:lat, :lon, :timezone, :timezone_offset,
                       :dt, :sunrise, :sunset, :temp, :feels_like, :pressure, :humidity, 
                       :dew_point, :uvi, :clouds, :visibility, :wind_speed, :wind_gust, :wind_deg, 
                       :id, :main, :description, :icon, :rain, :snow)''', data
                       )
        
        conn.commit()

In [184]:
extract_load(get_links())

In [185]:
df = pd.read_sql_query('''SELECT * FROM weather''', conn)
df

,lat,lon,timezone,timezone_offset,date_time,sunrise,sunset,temp,feels_like,pressure,...,visibility,wind_speed,wind_gust,wind_deg,weather_id,main,description,icon,rain,snow
0,51.5072,0.1276,Europe/London,0,1708581600,1708585263,1708622726,9.86,8.29,989,...,10000,3.09,N/A,190,803,Clouds,broken clouds,04n,N/A,N/A
1,51.5072,0.1276,Europe/London,0,1708585200,1708585263,1708622726,9.87,7.79,988,...,10000,4.12,N/A,190,804,Clouds,overcast clouds,04n,N/A,N/A
2,51.5072,0.1276,Europe/London,0,1708588800,1708585263,1708622726,10.24,9.70,986,...,10000,5.66,N/A,170,803,Clouds,broken clouds,04d,N/A,N/A
3,51.5072,0.1276,Europe/London,0,1708592400,1708585263,1708622726,10.94,10.44,984,...,10000,5.14,N/A,170,803,Clouds,broken clouds,04d,N/A,N/A
4,51.5072,0.1276,Europe/London,0,1708596000,1708585263,1708622726,11.63,11.20,982,...,10000,4.63,9.77,170,500,Rain,light rain,10d,0.24,N/A
5,51.5072,0.1276,Europe/London,0,1708599600,1708585263,1708622726,11.95,11.55,980,...,4900,5.66,N/A,200,500,Rain,light rain,10d,0.18,N/A
6,51.5072,0.1276,Europe/London,0,1708603200,1708585263,1708622726,12.13,11.70,978,...,10000,6.69,N/A,210,803,Clouds,broken clouds,04d,N/A,N/A
7,51.5072,0.1276,Europe/London,0,1708606800,1708585263,1708622726,9.29,7.07,977,...,10000,4.12,N/A,280,803,Clouds,broken clouds,04d,N/A,N/A
8,51.5072,0.1276,Europe/London,0,1708610400,1708585263,1708622726,8.75,7.68,975,...,10000,2.06,N/A,250,500,Rain,light rain,10d,0.42,N/A
9,51.5072,0.1276,Europe/London,0,1708614000,1708585263,1708622726,7.66,3.57,975,...,6000,8.23,N/A,280,501,Rain,moderate rain,10d,1.54,N/A
